In [28]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score

s3 = boto3.resource('s3')
bucket_name = "rachaeld-data445"
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [29]:
# removing observations with NA
heart = heart.dropna()

In [30]:
# defining the input and target variables
X = heart[['male','age', 'currentSmoker','totChol', 'BMI', 'heartRate','sysBP', 'glucose']]
Y = heart['TenYearCHD']

# splitting the data
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = .2)

In [31]:
# min-max transformation
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [32]:
# identifying important variables with LASSO
lasso_md = LassoCV().fit(X_train, Y_train)

# extracting the best lambda
lasso_lambda = lasso_md.alpha_

# fitting LASSO with the best lambda
lasso_md = Lasso(alpha = lasso_lambda).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.07219438,  0.26528289,  0.0386707 ,  0.        , -0.        ,
        0.        ,  0.45555516,  0.31076789])

In [34]:
# defining the input and target variables for logistic
X_train_logit = X_train[['age', 'BMI','totChol','sysBP', 'glucose']]
X_test_logit = X_test[['age', 'BMI','totChol','sysBP', 'glucose']]

#buidling the logistic regression model
logit_md = LogisticRegression().fit(X_train_logit, Y_train)

# predicting on test
logit_pred = logit_md.predict_proba(X_test_logit)[:,1]

#changing liklihoods to labels
logit_label = np.where(logit_pred < .1, 0,1)
logit_label

array([0.11872104, 0.21864085, 0.3892918 , 0.07520708, 0.10168532,
       0.13344076, 0.08078045, 0.05917991, 0.08890076, 0.29189294,
       0.08764571, 0.11311065, 0.08456115, 0.1113654 , 0.16930687,
       0.0838174 , 0.08543188, 0.13233617, 0.24709849, 0.16384259,
       0.07521747, 0.38113128, 0.13947447, 0.08063694, 0.37536868,
       0.18311004, 0.07455568, 0.12731193, 0.08931158, 0.07070476,
       0.13971167, 0.30124132, 0.16900474, 0.15991604, 0.09122705,
       0.08568754, 0.07191714, 0.32905622, 0.17496603, 0.2382844 ,
       0.25672423, 0.05635188, 0.2813731 , 0.21882383, 0.24569749,
       0.13491216, 0.35889164, 0.53434245, 0.17321169, 0.07655222,
       0.1694566 , 0.08946165, 0.10560532, 0.20004697, 0.14980447,
       0.07074435, 0.1023182 , 0.13188313, 0.20928241, 0.27339694,
       0.086333  , 0.21485775, 0.23996278, 0.14678649, 0.09498096,
       0.05599334, 0.33381411, 0.18922699, 0.12462576, 0.15191354,
       0.15387675, 0.09885326, 0.53219031, 0.1333998 , 0.07401